In [2]:
import mysql.connector

try:
    conn = mysql.connector.connect(host='localhost',user='root',port=3307,password='inix2025')
    print("Connected")
except:
    print("Connection failed")
#conn.close()
cursor = conn.cursor()

Connected


In [16]:
import pandas as pd
def tampil_data():
    rows = cursor.fetchall()
    columns = [col[0] for col in cursor.description]

    df = pd.DataFrame(rows, columns=columns)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    return df

In [4]:
cursor.execute("create database toko")

In [5]:
cursor.execute("use toko")

In [6]:
cursor.execute("select database()")
tampil_data()

,database()
0,toko


# A. Integritas Entitas
- Tujuan: Memastikan bahwa setiap baris dalam tabel dapat diidentifikasi secara unik.
- Implementasi: **`PRIMARY KEY`**

In [7]:
cursor.execute("""
CREATE TABLE pelanggan(
  id INT PRIMARY KEY,
  nama VARCHAR(50)
)
""")

In [8]:
cursor.execute("describe pelanggan")
tampil_data()

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,
1,nama,varchar(50),YES,,None,


### Tidak boleh ada dua pelanggan dengan id yang sama, dan kolom id otomatis jadi **`NOT NULL`**.
---

# B. Inetgriras Referensial
- Tujuan: Menjaga konsistensi hubungan antar tabel (foreign key).
- Implementasi: FOREIGN KEY

In [9]:
cursor.execute(""" 
CREATE TABLE pesanan(
  id INT PRIMARY KEY,
  pelanggan_id INT,
  FOREIGN KEY (pelanggan_id) REFERENCES pelanggan(id)
)
""")

In [18]:
cursor.execute("describe pesanan")
tampil_data()

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,
1,pelanggan_id,int,YES,MUL,None,


In [17]:
cursor.execute("SHOW CREATE TABLE pesanan")
tampil_data()

,Table,Create Table
0,pesanan,"CREATE TABLE `pesanan` (\n `id` int NOT NULL,\n `pelanggan_id` int DEFAULT NULL,\n PRIMARY KEY (`id`),\n KEY `pelanggan_id` (`pelanggan_id`),\n CONSTRAINT `pesanan_ibfk_1` FOREIGN KEY (`pelanggan_id`) REFERENCES `pelanggan` (`id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci"


### Tidak bisa membuat pesanan dengan pelanggan_id yang tidak ada di tabel pelanggan.
---

# C. Integritas Domain (Validasi Nilai)
- Tujuan: Memastikan bahwa data yang dimasukkan sesuai dengan tipe dan aturan nilai yang diperbolehkan.
- Implementasi:
```mysql
NOT NULL
UNIQUE
CHECK
ENUM
SET
DEFAULT
```

In [19]:
cursor.execute("""
CREATE TABLE produk (
  id INT PRIMARY KEY,
  nama VARCHAR(100) NOT NULL,
  harga DECIMAL(10,2) CHECK (harga > 0),
  kategori ENUM('Elektronik', 'Pakaian', 'Makanan'),
  stok INT DEFAULT 0
)
""")

In [21]:
cursor.execute("describe produk")
tampil_data()

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,
1,nama,varchar(100),NO,,None,
2,harga,"decimal(10,2)",YES,,None,
3,kategori,"enum('Elektronik','Pakaian','Makanan')",YES,,None,
4,stok,int,YES,,0,


In [23]:
cursor.execute("show create table produk")
tampil_data()

,Table,Create Table
0,produk,"CREATE TABLE `produk` (\n `id` int NOT NULL,\n `nama` varchar(100) NOT NULL,\n `harga` decimal(10,2) DEFAULT NULL,\n `kategori` enum('Elektronik','Pakaian','Makanan') DEFAULT NULL,\n `stok` int DEFAULT '0',\n PRIMARY KEY (`id`),\n CONSTRAINT `produk_chk_1` CHECK ((`harga` > 0))\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci"


- **`NOT NULL`**: Nama produk wajib diisi.
- **`CHECK`**: Harga harus lebih dari 0.
- **`ENUM`**: Hanya kategori tertentu yang bisa dipilih.
- **`DEFAULT`**: Jika stok tidak ditentukan, maka diset ke 0.
---

# D. Integrasi Unik
- Tujuan: Mencegah duplikasi data pada kolom tertentu.
- Implementasi: **`UNIQUE`**

In [24]:
cursor.execute("""
CREATE TABLE pengguna (
  id INT PRIMARY KEY,
  email VARCHAR(100) UNIQUE
);
""")

In [25]:
cursor.execute("show create table pengguna")
tampil_data()

,Table,Create Table
0,pengguna,"CREATE TABLE `pengguna` (\n `id` int NOT NULL,\n `email` varchar(100) DEFAULT NULL,\n PRIMARY KEY (`id`),\n UNIQUE KEY `email` (`email`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci"


In [26]:
cursor.execute("describe pengguna")
tampil_data()

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,
1,email,varchar(100),YES,UNI,None,


## Tidak boleh ada dua pengguna dengan alamat email yang sama.
---

# E. Skenario

| Skenario                            | Tanpa Integritas | Dengan Integritas       |
| ----------------------------------- | ---------------- | ----------------------- |
| User menginput email dua kali       | Bisa duplikat    | Ditolak (`UNIQUE`)      |
| Produk dengan harga minus           | Bisa disimpan    | Ditolak (`CHECK`)       |
| Order dari pelanggan yang tidak ada | Bisa masuk       | Ditolak (`FOREIGN KEY`) |
| Data penting kosong                 | Bisa kosong      | Ditolak (`NOT NULL`)    |

- **Data valid**: tidak asal-asalan.
- **Data konsisten**: antar tabel saling sinkron.
- **Data unik**: tidak ganda untuk entitas penting.
- **Data terhubung**: relasi antar tabel dijaga.
